In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -U d2l    # install d2l library

In [ ]:
import torch
from torch import nn
from d2l import torch as d2l

<p style="text-align:center;">
    <img src="https://d2l.ai/_images/gan.svg"/>
    <br>
    Generative Adversarial Networks
</p>

In [ ]:
import argparse
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

print(os.listdir("../input/assignment/celeba_tiny"))
#dataroot = '../input/celeba-dataset'
dataroot = '../input/assignment'
workers = 8
batch_size = 128
image_size = 64
nc = 3 # number of channels
nz = 100 # size of z latent vector
ngf = 64 # size of feature maps in generator
ndf = 64 # size of feature maps in discriminator
num_epochs = 10
lr = 0.0002
beta1 = 0.5
ngpu = 1 # number of GPUs available, 0: CPU

In [ ]:
dataset = dset.ImageFolder(root = dataroot,
                          transform = transforms.Compose([
                              transforms.Resize(image_size),
                              transforms.CenterCrop(image_size),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                          ]))

dataloader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, 
                                        shuffle = True, num_workers = workers)

device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

real_batch = next(iter(dataloader))
plt.figure(figsize = (8, 8))
plt.axis("off")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64],
                                        padding = 2, normalize=True).cpu(), (1, 2, 0)))


In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(            
            # nz = 100, ngf = 64, nc = 3 
            nn.ConvTranspose2d(in_channels = nz, out_channels = ngf * 8, 
                               kernel_size = 4, stride = 1, padding = 0, bias = False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # output: (512, 4, 4)
            nn.ConvTranspose2d(in_channels = ngf * 8, out_channels = ngf * 4, 
                               kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # output: (256, 8, 8)
            nn.ConvTranspose2d(in_channels = ngf * 4, out_channels = ngf * 2, 
                               kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # output: (128, 16, 16)
            nn.ConvTranspose2d(in_channels = ngf * 2, out_channels = ngf, 
                               kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # output: (64, 32, 32)
            nn.ConvTranspose2d(in_channels = ngf, out_channels = nc, 
                               kernel_size = 4, stride= 2, padding = 1, bias = False),            
            nn.Tanh()            
            # output: (3, 64, 64)
        )
        
    def forward(self, input):
        return self.main(input)

In [ ]:
net_g = Generator(ngpu).to(device)
net_g.apply(weights_init)

print(net_g)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # nc = 3, ndf = 64
            nn.Conv2d(in_channels = nc, out_channels = ndf, 
                      kernel_size = 4, stride = 2, padding = 1, bias = False),            
            nn.LeakyReLU(0.2, inplace = True),            
            # output: (64, 32, 32)
            nn.Conv2d(in_channels = ndf, out_channels = ndf * 2, 
                      kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(ndf * 2),            
            nn.LeakyReLU(0.2, inplace = True),
            # output: (128, 16, 16)
            nn.Conv2d(in_channels = ndf * 2, out_channels = ndf * 4, 
                      kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace = True),
            # output: (256, 8, 8)
            nn.Conv2d(in_channels = ndf * 4, out_channels = ndf * 8, 
                      kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace = True),
            # output: (512, 4, 4)
            nn.Conv2d(in_channels = ndf * 8, out_channels = 1, 
                      kernel_size = 4, stride = 1, padding = 0, bias = True),
            nn.Sigmoid()
        )
        
    def forward(self, input):
        return self.main(input)

In [ ]:
net_d = Discriminator(ngpu).to(device)
net_d.apply(weights_init)

print(net_d)

In [ ]:
criterion = nn.BCELoss() # Binary Cross Entropy Loss

fixed_noise = torch.randn(64, nz, 1, 1, device=device) # nz = 100

real_label = 1.0
fake_label = 0.0

# lr = 0.0002, beta1 = 0.5
optimizer_d = optim.Adam(net_d.parameters(), lr = lr, betas = (beta1, 0.999))
optimizer_g = optim.Adam(net_g.parameters(), lr = lr, betas = (beta1, 0.999))

In [ ]:
import time

In [ ]:
img_list = []
g_losses = []
d_losses = []
iters = 0

start_time = time.time()

for epoch in range(num_epochs):
    d_x, d_g_z1, d_g_z2 = 0., 0., 0.
    err_d, err_g = torch.Tensor(), torch.Tensor()
    
    for i, data in enumerate(dataloader, 0):
        # 1. update discriminator
        # train real data
        net_d.zero_grad()
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0) # batch_size = 128
        label = torch.full((b_size, ), real_label, dtype = torch.float, device = device)
        output = net_d(real_cpu).view(-1)
        err_d_real = criterion(output, label)
        err_d_real.backward()
        d_x = output.mean().item()
                
        # train fake data
        noise = torch.randn(b_size, nz, 1, 1, device = device) # (128, 3, 1, 1)
        fake = net_g(noise)
        label.fill_(fake_label)
        
        # classify fake data
        output = net_d(fake.detach()).view(-1)
        
        # calculate net_d loss on fake data
        err_d_fake = criterion(output, label)
        err_d_fake.backward()
        d_g_z1 = output.mean().item()        
        
        # compute error of net_d as real and fake 
        err_d = err_d_real + err_d_fake
        
        # update net_d
        optimizer_d.step()
        
        # 2. update generator
        net_g.zero_grad()
        label.fill_(real_label)
        
        # discriminate fake data after updated net_d
        output = net_d(fake).view(-1)
        
        # calculate net_g loss on fake data
        err_g = criterion(output, label)
        err_g.backward()
        d_g_z2 = output.mean().item()
        
        # update net_g
        optimizer_g.step()
        
        g_losses.append(err_g.item())
        d_losses.append(err_d.item())
        
        if (iters % 100 == 0) or ((epoch == num_epochs - 1) and (i == len(dataloader) - 1)):
            with torch.no_grad():
                fake = net_g(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding = 2, normalize = True))

        iters += 1
    print("{} / {} epochs, loss_d: {:.4f}, loss_g: {:.4f}, D(x): {:.4f}, D(G(z)): {:.4f} / {:.4f})".format(
        epoch + 1, num_epochs, err_d.item(), err_g.item(), d_x, d_g_z1, d_g_z2))
        
print("run time: {:.2f} s".format(time.time() - start_time))

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(g_losses,label="G")
plt.plot(d_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
from IPython.display import HTML

fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

#plt.imshow(np.transpose(img_list[-1],(1,2,0)))

HTML(ani.to_jshtml())

In [ ]:
real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(13,13))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()